In [1]:
import os
import re
import time
import pandas as pd
from openai import OpenAI

In [2]:
products_df = pd.read_csv("data/labeled_tools_and_home_improvement_products.csv")

In [3]:
products_df.head()

,id,title,description,label
0,B09M8J9LQ9,Decute 200LED Christmas String Lights Outdoor ...,Specifications Light Color Multicolored Applic...,22
1,B0BNZ888C7,Upgrade Hydro Shower Jet Head High Pressure Hy...,HighPressure Filtered Shower Head with 3 Repla...,10
2,B07TLCQ3NR,"1"" Adhesive Furniture Sliders -Furnigear Chair...",Adhesive Furniture Glides SlidersFurnigear Hea...,17
3,B09HRSKRFB,"Sunlite 40455 LED A19 Colored Light Bulb, 3 Wa...",Sunlites colored decorative A19 household ligh...,3
4,B014AV8FY2,Dixie Belle Paint Company Chalk Finish Furnitu...,CHALK MINERAL PAINT Amethyst is a rich deep m...,9


In [18]:
# We will sample 3 products per group and create sample queries for each of them
product_samples = products_df.groupby('label').sample(n=5, random_state=234).reset_index(drop=True)
product_samples.head(n=12)

,id,title,description,label
0,B08LBHKCG6,ESTWING Rock Pick - 22 oz Geological Hammer wi...,Product Description Estwing Rock Picks are the...,-1
1,B08X4LDHS7,"SOAIY Star Projector, Night Light 3IN1 Starry ...",Galaxy Light Projector Smart LifeWorks with Al...,-1
2,B0BG3ZNVTX,Christmas Decorative Laser Lights Projector Ou...,OUTDOOR USE Longdistance projection is still ...,-1
3,B000JFLMBM,DANCO Brass Closet Bolts with Nuts and Washers...,Product Description Danco is one of the larges...,-1
4,B09KC9BG3D,"Moon Lamp, LOGROTATE 16 Colors LED Night Light...",A Unique Kids Night Light Moon lamp with the d...,-1
5,B0B2RPLQKS,"Cordless Screwdriver, VIGRUE Rechargeable Elec...",Mutifunction Drill Driver and Electric Screwdr...,0
6,B0C49QGX4L,Dremel Lite 7760 N/10 4V Li-Ion Cordless Rotar...,Dremel Lite 7760 N10 4V LiIon Cordless Rotary ...,0
7,B077ZYMK1W,Milwaukee Electric Tools MLW2553-20 M12 Fuel 1...,The M12 FUEL 14 in Hex impact driver once agai...,0
8,B0BWN8YQLT,Winzwon Cutting Wheels Set 44 Pcs for Dremel R...,Diamond Cutting Wheels 15 Pcs Match with 2pcs ...,0
9,B00Z82EYZ2,Ridgid R840095 Gen5X Genuine OEM Dual Chemistr...,If you have Ridgid 18V batteries youll want to...,0


In [5]:
openai_api_key = os.environ['OPENAI_API_KEY']

In [6]:
class QueryGenerator:
    def __init__(self, api_key):
        """Initialize the generator with API key and standard prompts"""
        self.client = OpenAI(api_key=api_key)
        
        # Store the system and example prompts
        self.system_prompt = """You are an expert in e-commerce search optimization. Given a product description, generate a list of 5 realistic search queries a customer might use to find the product, even if the customer does not know the exact name of the product they are looking for.

        Guidelines:
        - Keep queries short (2-10 words)
        - Avoid exact repetition of product specs; focus on what a customer would type
        - Include a mix of general, descriptive, and feature-focused queries
        - Consider different use cases for each product (e.g. baking soda can be used for cooking, cleaning, and removing bad odors).
        - Use casual, natural language
        - Return ONLY the list of 5 queries enclosed in [] and separated by ',' with no additional text"""

        self.example_prompt = """Here are some examples:

        Product Description: "Desk Lamps with Smooth Dimming Function from 100 to 0The rotary knob on the lamp base allows you to set the brightness to any level from 0 to 100 Easy to change the visual appearance and mood of your space Bedside Table Lamps with 2 USB Charging Ports and 1 AC OutletEach lamp comes with two USB charging ports5V21A and one 2prong AC power outlet120V which are available for use whether the lights on or off Super convenient for you to charge your cellphone iPad Kindle diffuser Apple Watch tablet and other electronic devices Save you from the trouble of not having enough wall sockets to charge multiple devices at the same time EyeCaring LED Edison Bulbs IncludedTwo vintage LED bulbs are included in the package so you can have two workable table lamps instantly upon receiving Save your time in finding compatible ones These 60Watt equivalent LED bulbs offer brightness up to 800LM by drawing only 7W energy The 5000K light these bulbs emit is similar to natural daylight making your surrounding clear and helping you stay focused Minimalist Upright DesignFeaturing clearline metal body in black finish and seeded glass shade this table lamp complements any decor styles from industrial modern antique to midcentury Ideal light source for all areas like bedroom living room kitchen office kids room nursery room and guestroom Lovely Gift Idea Perfect gift choice for family and friends on festivals birthdays graduation and housewarming days"
        
        Queries:
        [adjustable desk lamp, light with charging port for devices, light for working on my desk, basic design lamp, multipurpose lamp]
        
        Product Description: "Keyless Entry Door Lock SMONET Smart Lock can recognize your fingerprint in just 05 seconds and unlock your door in 1 second faster than fumbling for your keys Compatible with Alexa or Google Assistant Requires a wifi gateway to hook it up to Internet sold separately Smart Bluetooth Keypad Lock Not only bring you security and trust but also bring you a brandnew smart home Five ways to unlock your door  Mobile Keypad Fingerprint IC Fob and Mechanical Keys Customize Management  Sign in Bluetooth Locks for Front Door you could check the records about unlocking and wrong passcode in real time Code Door Lock allow setting generate Timed Permanent Onetime or Customized passcode for guests friends housekeepers or employees Smart Locks for Front Door Intuitive touchscreen display alloy body long battery life 4pcs AA batteries can provide 10000 openings approx 12 months For security reason it will automatically lock for 5 minutes when enter the password incorrectly more than 5 times A great solution for home hotel Airbnb etc Excellent Customer Service  1year warranty and free lifetime technical support Any question such as installation or operation problemsor for more gateway information please feel free to contact us"
        
        Queries:
        [I want a more secure lock for my front door, lock and unlock door with my phone, open door with my finger, door lock that doesn't use key, fancy tech door lock]
        """

    def generate_single(self, product_description):
        """Generate queries for a single product description"""
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": self.example_prompt},
                    {"role": "user", "content": f"Now generate queries for this product description: {product_description}"}
                ],
                temperature=0.7,
                max_tokens=50
            )
            
            res = response.choices[0].message.content
            return res.replace('[', '').replace(']', '').split(', ')
            
        except Exception as e:
            print(f"Error generating queries: {e}")
            return []

    def generate_batch(self, product_descriptions, batch_size: int = 5):
        """
        Generate queries for multiple products efficiently.
        Uses batching and includes rate limiting.
        """
        batch_results = []
        
        # Process in batches
        for i in range(0, len(product_descriptions), batch_size):
            batch = product_descriptions[i:i + batch_size]
            
            # Create a single prompt for the batch
            batch_prompt = "Generate queries for each of these products. For each product, start with 'Product X:' where X is the product number:\n\n"
            for idx, desc in enumerate(batch, 1):
                batch_prompt += f"Product {idx}: {desc}\n\n"
            
            try:
                response = self.client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": self.system_prompt},
                        {"role": "user", "content": self.example_prompt},
                        {"role": "user", "content": batch_prompt}
                    ],
                    temperature=0.7,
                    max_tokens=50 * len(batch)  # Adjust tokens based on batch size
                )
                
                # Parse the batch response
                response_text = response.choices[0].message.content
                response_list = re.sub("Product \d: ", "", response_text).replace('[', '').replace(']', '').split('\n')
                batch_results.extend([text.split(', ') for text in response_list])
                # batch_results.append(response_text)
                                
                # Rate limiting - wait 1 second between batches
                if i + batch_size < len(product_descriptions):
                    time.sleep(5)
                    
            except Exception as e:
                print(f"Error processing batch starting at index {i}: {e}")
                continue
        
        return batch_results

In [7]:
generator = QueryGenerator(openai_api_key)

In [19]:
# Number of products for which we are generating queries
product_samples['description'].shape

(295,)

In [9]:
# Create example queries for all descriptions
results = generator.generate_batch(product_samples['description'])

In [51]:
len(results)

279

There is some sort of mismatch between the number of product descriptions and the number of results returned. We will have to debug this manually to figure out which products did not get sample queries returned to properly align the results. We will do this in another notebook. For now, let's just export the queries so that we can save them for later.

In [44]:
# product_samples.loc[:, ['query_1', 'query_2', 'query_3', 'query_4', 'query_5']] = results

In [48]:
sample_queries = pd.DataFrame(results)
sample_queries.columns = ['query_1', 'query_2', 'query_3', 'query_4', 'query_5']

In [49]:
sample_queries.head()

,query_1,query_2,query_3,query_4,query_5
0,geologist rock hammer,durable rock pick with leather grip,American made geological tool,rock hammer with leatherwrapped handle,versatile rock pick
1,galaxy light projector with Alexa,star and ocean wave projector,remote control starry night light,voice control galaxy projector,WiFi controlled star projector
2,long distance Christmas laser lights,outdoor laser projector with remote control,weatherproof holiday light projector,easy to install outdoor light projector,professional laser lights for Christmas
3,toilet bowl anchor bolts,DIY toilet bowl fasteners,brass closet bolts for toilet,secure toilet bowl to floor,easy installation toilet bolts
4,kids moon lamp night light,3D printed moon light for nursery,remote control moon lamp,adjustable brightness moon light,decorative moon light for kids


In [50]:
sample_queries.to_csv('data/home_products_with_sample_queries', index=False)